In [1]:
#import kagglehub

"""
# Download latest version
path = kagglehub.dataset_download("Path to working directory", "dataset_name") # Esto es para descartar el dataset inicial, poner el path en tu ordenador local

print("Path to dataset files:", path)
"""

'\n# Download latest version\npath = kagglehub.dataset_download("Path to working directory", "dataset_name") # Esto es para descartar el dataset inicial, poner el path en tu ordenador local\n\nprint("Path to dataset files:", path)\n'

In [2]:
import pandas as pd
df = pd.read_csv('amazon_reviews.csv')
df.head() #Aqui podemos ver el dataset usado, varias columnas innecesarias


,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [3]:
#Limpiar el dataset
dfSimple = df.drop(columns=['reviewerName', 'reviewTime', 'day_diff', 'helpful_yes', 'helpful_no', 'total_vote', 'score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound'])

#De inicio trabajaremos con un dataset masivamente simplificado para poder hacer pruebas, la columna unnamed:0 no se elimina porque es un identificador de la fila y puede ser util mas adelante
dfSimple.dropna(subset=["reviewText"], inplace=True)
dfSimple.head()


,Unnamed: 0,overall,reviewText
0,0,4.0,No issues.
1,1,5.0,"Purchased this for my device, it worked as adv..."
2,2,4.0,it works as expected. I should have sprung for...
3,3,5.0,This think has worked out great.Had a diff. br...
4,4,5.0,"Bought it with Retail Packaging, arrived legit..."


In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [5]:
# NLTK resources
nltk.download('stopwords') #Llista de stopwords + comunes
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minaa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\minaa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Cleaning function : punctuation, numbers, URL, convert to lowercase, tokenize, remove stopwords.
"""
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    #words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in text if word not in stop_words]
    return ' '.join(words)
"""

def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Split text into words
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Join words back into a single string
    return ' '.join(words)

In [7]:
# Create a new DataFrame with cleaned reviewText and the overall column
dfCleaned = dfSimple.copy()
dfCleaned['reviewText'] = dfCleaned['reviewText'].apply(clean_text)  # Apply the cleaning function
#dfCleaned['overall'] = dfCleaned['overall']  # Include the overall column

print(dfCleaned.head())

   Unnamed: 0  overall                                         reviewText
0           0      4.0                                             issues
1           1      5.0  purchased device worked advertised never much ...
2           2      4.0  works expected sprung higher capacity think ma...
3           3      5.0  think worked greathad diff bran gb card went s...
4           4      5.0  bought retail packaging arrived legit orange e...


In [8]:
# Save the cleaned dataset 
dfCleaned.to_csv('amazon_reviews_Cleaned.csv', index=False)

print("Cleaned dataset saved as 'amazon_reviews_Cleaned.csv'")

Cleaned dataset saved as 'amazon_reviews_Cleaned.csv'


In [9]:
#Por ahora, trabajaremos con valores de 0 o 1 en la columna overall para tener un problema simple y poder de inicio identifcar de manera binaria si es positiva o negativa la review. Despues, querremos predecir la nota de la review por el texto con mas exactitud
dfSimpleBinary = dfCleaned.copy()
dfSimpleBinary['overall'] = dfSimpleBinary['overall'].apply(lambda x: 1 if x > 2.5 else 0)
dfSimpleBinary['overall'].value_counts()

dfSimpleBinary.head(1000)

,Unnamed: 0,overall,reviewText
0,0,1,issues
1,1,1,purchased device worked advertised never much ...
2,2,1,works expected sprung higher capacity think ma...
3,3,1,think worked greathad diff bran gb card went s...
4,4,1,bought retail packaging arrived legit orange e...
...,...,...,...
996,996,1,bought digital recorder memory card slot card ...
997,997,1,much say memory card works great go pro hero s...
998,998,1,gig microsd card costs less dollar per gig cla...
999,999,1,im nerd hoarder waited year memory gb wanted c...


In [10]:
#Guardamos el dataset limpio para no sobreescribirlo accidentalmente, este paso es solo una precaucion. solo falta borrar los nans
dfSimpleBinary.to_csv('amazon_reviews_simpleBinary.csv', index=False)

In [11]:
#applyCleantext

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Apply TF-IDF
tfidf = TfidfVectorizer(max_features=5000)  # Limit to 5000 features for simplicity
X = tfidf.fit_transform(dfSimpleBinary['reviewText'])  

# y = Etiqueta
y = dfSimpleBinary['overall']  

In [13]:
# Això nomes per ara per visualitzar el resultat despres de TF-IDF. X : sparse matrix (4914,5000), sparse matrix es representa per coordenades ja que en gran part està buida
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 104460 stored elements and shape (4914, 5000)>
  Coords	Values
  (0, 1811)	1.0
  (1, 2960)	0.2390691997557985
  (1, 901)	0.2500956641441643
  (1, 4814)	0.22041352975002462
  (1, 65)	0.3079511880305262
  (1, 2283)	0.23746924750824877
  (1, 2235)	0.22880268690799568
  (1, 2527)	0.15894729456975054
  (1, 2124)	0.16342700659146378
  (1, 3614)	0.2634346235438177
  (1, 980)	0.3515768967137597
  (1, 2004)	0.25280027831644336
  (1, 4010)	0.3197017006470681
  (1, 371)	0.4658817477827855
  (2, 4826)	0.10167611425771444
  (2, 1187)	0.21026831614423724
  (2, 3834)	0.4017209024878068
  (2, 1617)	0.26622741443620596
  (2, 460)	0.18410928375398103
  (2, 4235)	0.20417737012953208
  (2, 2031)	0.22707996152754703
  (2, 320)	0.22808251479726946
  (2, 1032)	0.32830745830827224
  (2, 4490)	0.2923424981405928
  (2, 2453)	0.37010345303225173
  :	:
  (4912, 3723)	0.28806210890236555
  (4912, 1308)	0.4263389161515658
  (4912, 3453)	0.305510277334716

In [14]:
print(y)

0       1
1       1
2       1
3       1
4       1
       ..
4910    0
4911    1
4912    1
4913    1
4914    1
Name: overall, Length: 4914, dtype: int64


In [15]:
import pandas as pd

# Cargar el dataset limpio
dfCleaned = pd.read_csv('amazon_reviews_Cleaned.csv')

# Crear el dataset ternario
dfTernary = dfCleaned.copy()
dfTernary['overall'] = dfTernary['overall'].apply(lambda x: 0 if x < 3 else (1 if x == 3 else 2))

# Verificar la distribución de las etiquetas
print(dfTernary['overall'].value_counts())

# Guardar el nuevo dataset
dfTernary.to_csv('amazon_reviews_ternary.csv', index=False)

print("Dataset ternario guardado como 'amazon_reviews_ternary.csv'")

2    4448
0     324
1     142
Name: overall, dtype: int64
Dataset ternario guardado como 'amazon_reviews_ternary.csv'
